In [1]:
#!/bin/bash
## This is similar to cron_test.sh but is not called with sbatch, 
## Because it runs sbatch itself
## It interacts with the sqlite3 db called 
## /global/cfs/cdirs/desi/science/td/daily-search/transients_search.db
## Blame Antonella Palmese version Jan 2021 for the ugliness of this code
## First draft reworking to fix db reading issues with DESIDIFF
## Many ??
## 06/04/21 11:56:00 Cleo Lepart

### set -e
echo `date` Running daily time domain pipeline on `hostname`
#- Configure desi environment if needed
if [ -z "$DESI_ROOT" ]; then
    echo "Loading DESI modules"
    module use /global/common/software/desi/$NERSC_HOST/desiconda/startup/modulefiles
    echo "module load"
    module load desimodules/master
fi

tiles_path="/global/project/projectdirs/desi/spectro/redux/daily/tiles" 

run_path="/global/cscratch1/sd/akim/project/timedomain/cronjobs/" 

td_path="/global/cfs/cdirs/desi/science/td/daily-search/" 

SyntaxError: invalid syntax (<ipython-input-1-b38cb915510d>, line 12)

In [2]:
##################
#Now double check that we have run over all new exposures. If not, we send new runs.
#The database is updated in the next lines starting where it calls python ${run_path}exposure_db.py
#That script updates the exposures table, which is later compared to the processed exposures in 
#desitrip_exposures to find unprocessed exposures. 
#desitrip_exposures is then updated with new exposures that went through the classifier in the classifier script
#ATM this only works for desitrip outputs - needs to be added to specdiff

echo "Looking for new exposures"


python ${run_path}exposure_db.py daily #creates exposures db cf. below from transients_search.db cf. /cronjobs/exposure_db.py L162 ?

query="select distinct obsdate,tileid from exposures where (tileid,obsdate) not in (select tileid,obsdate from desidiff_H_coadd_exposures);" 
# query="select distinct obsdate,tileid from exposures
# where (tileid,obsdate) not in (select tileid,obsdate from desidiff_cv_coadd_exposures);" 
mapfile -t -d $'\n' obsdates_tileids < <( sqlite3 ${td_path}transients_search.db "$query" ) #queries transients_search.db 


query_tileid_only = "select distinct tileid from exposures where (tileid,obsdate) not in (select tileid,obsdate from desidiff_H_coadd_exposures);" #AND obsdate>20210529;"
#queries transcients_search for unprocessed (new) tileids only

mapfile -t -d $'\n' tileids < < ( sqlite3 ${td_path}transients_search.db "$query_tileid_only" )
#arry of unprocessed tileids

SyntaxError: invalid syntax (<ipython-input-2-297624fa90d5>, line 9)

In [ ]:
desi_path="/global/cfs/cdirs/desi/science/td/db/"
#path needed for association of tileid (@transcients_search.db) to targetid (@desi.db)
#why not only use desi.db for this process? 

desi_query="select distinct TARGETID, YYYYMMDD from fibermap_daily;"
#command within for loop for fetching and insertion of targetid with associated tileid

for id in ${tileid[@]}
    mapfile -t targetid_arr < <( sqlite3 ${desi_path}desi.db "$query_desi")

